# Импорт и загрузка библиотек

In [123]:
!pip install --upgrade --no-cache-dir gdown
!gdown --id 11Q2g2aA-n7fYK95XEB3d6m0CWKSt6rob
!unzip quora_question_pairs_rus.csv.zip
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
!pip install wget
!pip install navec
!git clone https://github.com/facebookresearch/fastText.git
!pip3 install fastText/.
!pip install fasttext
!pip install fasttext
!pip install python-Levenshtein

/Users/vaceslavefimov/anaconda3/lib/python3.8/site-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=11Q2g2aA-n7fYK95XEB3d6m0CWKSt6rob
To: /Users/vaceslavefimov/Documents/ITMO/Data Analysis/quora_question_pairs_rus.csv.zip
100%|██████████████████████████████████████| 26.7M/26.7M [00:02<00:00, 9.41MB/s]
Archive:  quora_question_pairs_rus.csv.zip
replace quora_question_pairs_rus.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
--2022-02-28 14:37:56--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Sav

UnboundLocalError: local variable 'child' referenced before assignment

In [106]:
import pandas as pd
import numpy as np
from nltk.tokenize import WordPunctTokenizer
from navec import Navec
import fasttext
import nltk
from nltk.metrics import edit_distance

# Чтение данных

In [79]:
quora_df = pd.read_csv('quora_question_pairs_rus.csv', usecols=lambda x: 'Unnamed' not in x)
quora_df

,question1,question2,is_duplicate
0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1
...,...,...,...
404283,сколько ключевых слов есть на языке программир...,сколько ключевых слов есть в языке программиро...,0
404284,"ты веришь, что есть жизнь после смерти","правда ли, что есть жизнь после смерти",1
404285,что такое одна монета,что это за монета,0
404286,какова приблизительная годовая стоимость жизни...,"У меня проблема с волосами, но я хочу использо...",0


In [82]:
questions = []
questions.extend(quora_df.question1.values)
questions.extend(quora_df["question2"].values)
print(f'Всего вопросов: {len(questions)}')

Всего вопросов: 808576


In [47]:
path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

In [48]:
navec.get('python')

array([ 0.19214402, -0.1959362 ,  0.43695402, -0.33588752,  0.39676988,
       -0.07734875,  0.04260164,  0.67186844, -0.04072431, -0.07451263,
        0.49101624,  0.07942662,  0.20798121,  0.18426852, -0.30046153,
       -0.15204567,  0.36486024, -0.85307485,  0.48137093,  0.427026  ,
       -0.6845362 , -0.08942661, -0.3101123 , -0.45371357, -0.611568  ,
       -0.26479366, -0.9259754 , -0.11309931, -0.48673463,  0.42229277,
        0.20572862, -0.13167205, -0.8779028 ,  0.07185546,  0.02660668,
       -0.0868782 ,  0.572537  ,  0.6779026 , -0.11222838,  0.14546263,
       -0.03367202,  0.24061641,  0.08572771,  0.00990655,  0.25097123,
        0.01056161, -0.51034373,  0.09052686,  0.43652472,  0.3318286 ,
       -0.11107466, -0.14714225, -0.34816197, -0.3292252 ,  0.33444104,
       -0.07015625,  0.00249229,  0.03096647, -0.39550957, -0.37711838,
       -0.15093368, -0.3094792 ,  0.17217222,  0.21431988, -0.6753277 ,
       -0.5604233 ,  0.46426848,  0.08588701,  0.44196838,  0.34

In [128]:
def cosine(u, v):
    res = np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
    return res

In [131]:
cosine(navec['дом'], navec['здание'])

0.5017841

# Построение эмбеддингов

In [84]:
tokenizer = WordPunctTokenizer()


def embed(text):
    res = []

    tokens = tokenizer.tokenize(str(text).lower())

    for token in tokens:
        if token in navec:
            res.append(navec[token])

    if not res:
        res.append(navec['<unk>'])

    return np.mean(res, axis=0)

In [118]:
embeddings = [embed(question) for question in questions]


def get_top_n_questions(question, n=10):
    query = embed(question)
    cosines = [cosine(x, query) for x in embedded_data]

    top_questions = []
    for i in np.argpartition(cosines, -n)[-n:]:
        top_questions.append(embeddings[i])
        
    return top_questions

In [95]:
text = "Какой город самый большой в мире?"

get_top_n_questions(text, n=10)

какой город в мире самый красивый, чтобы жить в
какой лучший город в мире
какой лучший город в мире
какой самый большой океан в мире
какой самый большой океан в мире
какой самый большой океан в мире
какой самый большой город в мире
какой самый большой город в мире
какой самый большой город в мире по площади
какой самый большой город в мире


In [67]:
text = "Какой вуз самый лучший?"

get_result(text)

какой лучший университет
какой лучший университет
какой лучший колледж
какой лучший колледж
какой лучший dslr
какой лучший dslr
какой лучший колледж
какой лучший dslr
какой лучший метапоиска
какой лучший видеоредактор


In [68]:
text = "Какой драматург написал пьесу «Ромео и Джульетта»"

get_result(text)

Кто написал оперу Евгений Онегин?
Уильям Шекспир писал что-нибудь, кроме пьес
Уильям Шекспир писал что-нибудь, кроме пьес
Какой композитор написал первую советскую рок-оперу «Орфей и Эвридика»?
как написать сценарий комедии
Кто автор пьесы «Ромео и Джульетта»?
что написал Уильям Шекспир помимо пьес
какие пьесы сыграл соавтор шекспира
Какую оперу на сюжет «Ромео и Джульетты» написал композитор Винченцо Беллини?
что написал Уильям Шекспир помимо пьес


In [69]:
text = "Какие причины у глобального потепления?"

get_result(text)

какие убедительные доказательства существуют в отношении существования глобального потепления и изменения климата
какие возможные выгоды могут быть для глобального потепления
какие возможные выгоды могут быть для глобального потепления
каковы некоторые важные причины и следствия в отношении глобального потепления
каковы причины глобального потепления
каковы основные причины глобального потепления
каковы проблемы и последствия глобального потепления
какие последствия глобального потепления
каковы основные причины глобального потепления
каковы проблемы и последствия глобального потепления


In [70]:
text = "Какой сериал можно посмотреть на выходных?"

get_result(text)

где я могу посмотреть фильм на море
какой фильм я должен смотреть на netflix
какой фильм я должен смотреть на netflix
какой болливудский фильм лучше всего посмотреть в эти выходные
какой фильм стоит посмотреть на rustom или mohenjo daro
лучше смотреть на фурго, когда сериал видел фильм первым или нет
что такое ужасный фильм, который можно посмотреть на Хэллоуин
что такое ужасный фильм, который можно посмотреть на Хэллоуин
где я могу скачать или посмотреть сериал серии krishi
какой фильм я должен смотреть на netflix


In [71]:
text = "Кокой город самй бальшой?"

get_result(text)

Кем был основан город Нижний Новгород?
Бангалор - глобальный город
город Пуне, метро
Торонто опасный город
Мехико город опасен
Торонто - безопасный город
как город Бангалор
новый город Хэмптон
Сиэтл - хороший город
как город бангалор


In [119]:
question = "Какой город самый бальшой?"

most_similar_questions = get_top_n_questions(question)
most_similar_questions

[array([-3.36411446e-02, -1.67155445e-01, -9.03047621e-02,  1.37118101e-01,
        -3.88511084e-02,  2.96777580e-02,  1.59553692e-01,  4.53020781e-02,
         1.42547805e-02, -1.81781128e-01,  1.01213917e-01,  9.18569192e-02,
        -1.50566429e-01, -9.09952149e-02,  1.43377662e-01,  1.93421729e-02,
        -2.20950749e-02, -1.20526418e-01,  4.94207926e-02, -5.41270711e-02,
         2.21113097e-02, -3.14675033e-01,  1.69697747e-01,  2.93710232e-01,
         4.85514291e-03,  2.62212336e-01,  1.33413505e-02,  1.35894343e-01,
         1.01706915e-01,  7.67752975e-02, -1.73689984e-02, -5.46344258e-02,
         6.94844961e-01,  2.32098207e-01, -2.56335735e-02,  8.29735994e-02,
         1.56791583e-02, -1.51223823e-01,  2.92541713e-01,  8.09283704e-02,
         7.72744417e-02,  1.02106012e-01,  1.06005460e-01, -2.19693020e-01,
        -6.86194822e-02, -2.02667072e-01,  1.11503921e-01,  7.04794005e-02,
        -1.00854570e-02, -1.11726724e-01, -6.89589977e-03, -2.49512196e-01,
        -3.9

# Расстояние Левенштейна

In [125]:
map(lambda similar_text: edit_distance(question, similar_text), most_similar_questions)

NameError: name 'most_similar_questions' is not defined

In [126]:
edit_distance("климат", "климатический", transpositions=True)

7